In [ ]:
# !git clone https://github.com/dsa-playground/tvt2024.git
# %cd /content/tvt2024/
# !git pull
# !pip install -r requirements.txt -t "tvt2024"
# !pip install pyaml-env

In [1]:
# Imports
import warnings
import pandas as pd
from scripts.main import bekijk_data, kies_onderwerp, pas_modellen_toe, pas_gemiddelde_toe, pas_voortschrijdend_gemiddelde_toe, pas_lineaire_regressie_toe, pas_parameters_toe_en_evalueer, voorspel, onderzoek_afwijkingen, bereken_metrieken

## Settings
# settings for pandas
pd.set_option("display.max.columns",None) # alle kolommen tonen
pd.set_option("display.max.rows",500)    # eerste 500 rijen tonen
pd.set_option("display.precision", 2)     # precisie van de kolommen aanpassen
pd.set_option('display.float_format', lambda x: '{:.3f}'.format(x)) # floats output tot 3 decimalen
pd.set_option('display.max_colwidth', None)

# Disable warnings
warnings.filterwarnings('ignore')

# Inleiding

Jullie zijn allen werkzaam bij zorgorganisatie 'Zorgello' in de afdeling Analytics. De bestuurder komt binnen stormen:

Morgen heb ik een vergadering met de Raad van Commisarissen. Vorig jaar hebben ze me flink aan de tand gevoeld. Van een aantal zaken kon ik niet goed aangeven welke richting het op beweegt. Ik heb voor de vergadering morgen wat forecasts nodig. Kunnen jullie deze even snel maken? Het gaat om:
1. Het aantal clienten (ZZP >= 6)
2. Het ziekteverzuim van ons personeel
3. Kosten van inkoop materiaal 

Jullie als afdeling verdelen de onderwerpen. 

Forecasting: Predict or estimate (a future event or trend).





In [2]:
# a. Kies onderwerp
df = bekijk_data()



# VRAAG: Corona-periode invoegen? Als je dat doet, moet je ook een correctie 
# kunnen doorvoeren in de data om de effecten van corona te verminderen. OF... zeggen dat je tot een bepaalde periode niet gebruikt...


In [3]:
onderwerp = kies_onderwerp()


        Welke reeks wil je voorspellen (a, b, c):
                a. ZZP (verwachting clienten)
                b. Ziekteverzuimpercentage
                c. Inkoop materialen
                d. Flexpool (aantal personen)
        
Gekozen antwoord: Ziekteverzuim


In [4]:
# df_gemiddelde = pas_gemiddelde_toe(df=df, onderwerp=onderwerp)

# df_voortschrijdend_gemiddelde = pas_voortschrijdend_gemiddelde_toe(df=df, onderwerp=onderwerp)

# df_lineaire_regressie = pas_lineaire_regressie_toe(df=df, onderwerp=onderwerp, yearly_seasonality=False, weekly_seasonality=False)

In [5]:
df_voorspelling = pas_modellen_toe(df=df, onderwerp=onderwerp)

In [6]:
# Welk model is goed? Hoe evalueer je dit?
# Stap 4: Testen van je algoritme

# Plot twee grafieken:
# 1. Error van alle voorspellingen als timeseries voor testdata
# 2. Error als histogram / distributie voor testdata
onderzoek_afwijkingen(df=df_voorspelling, 
                      onderwerp=onderwerp, 
                      start='2023-05-15',
                      end=df_voorspelling.index.max()) # base_col, start en end zorgen dat deze uit de settings komen

In [7]:
# Introduceer metrieken
# Maak een tabel met de verschillende metrieken
# MASE, MAE, RMSE, MAPE, MDAPE
# Visualisatie van de metrieken?


# evalueer_resultaat()

bereken_metrieken(df=df_voorspelling, 
                  onderwerp=onderwerp, 
                  start='2023-05-15',
                  end=df_voorspelling.index.max())

De periode die wordt geanalyseerd is van 2023-05-15 tot 2024-04-13.


,Totale afwijking,Gemiddelde afwijking,Maximale afwijking,Mean Absolute Error,Mean Absolute Percentage Error,Root Mean Squared Error
Gemiddelde,-4.432,-0.013,1.453,0.587,0.070,0.689
Voortschrijdend gemiddelde,21.126,0.063,1.054,0.251,0.029,0.310
Lineaire regressie,-10.449,-0.031,1.432,0.585,0.070,0.688


In [8]:
# Stap 5: Optimaliseren van je algoritme

# Generiek voor alle modellen
vanaf_datum_train_periode = '2019-01-01'
tot_datum_train_periode = '2023-05-15'
vanaf_datum_test_periode = '2023-05-15'
tot_datum_test_periode = '2024-05-15'

# Gemiddelde
# - Geen specifieke parameters

# Voortschrijdend_gemiddelde
window_size = 7
shift_period = 365
predict=False

# Lineaire regressie
yearly_seasonality=True
weekly_seasonality=True
transformation='spline'
n_bins=14
strategy= 'uniform'
n_knots= 4
degree= 4

df_evalueer = pas_parameters_toe_en_evalueer(
    df=df,
    onderwerp=onderwerp,
    vanaf_datum_train_periode = vanaf_datum_train_periode,
    tot_datum_train_periode = tot_datum_train_periode,
    vanaf_datum_test_periode = vanaf_datum_test_periode,
    tot_datum_test_periode = tot_datum_test_periode,
    window_size= window_size,
    shift_period = shift_period,
    predict = predict,
    yearly_seasonality = yearly_seasonality,
    weekly_seasonality = weekly_seasonality,
    transformation=transformation, n_bins=n_bins, strategy=strategy, n_knots=n_knots, degree=degree
)

bereken_metrieken(df=df_evalueer, 
                  onderwerp=onderwerp, 
                  start=vanaf_datum_test_periode,
                  end=df_voorspelling.index.max())

De periode die wordt geanalyseerd is van 2023-05-15 tot 2024-04-13.


,Totale afwijking,Gemiddelde afwijking,Maximale afwijking,Mean Absolute Error,Mean Absolute Percentage Error,Root Mean Squared Error
Gemiddelde,-4.432,-0.013,1.453,0.587,0.070,0.689
Voortschrijdend gemiddelde,21.126,0.063,1.054,0.251,0.029,0.310
Lineaire regressie,65.933,0.197,0.964,0.269,0.031,0.337


In [9]:
# Fix fourth:
# Zorgen dat functie kan voorspellen

In [10]:
# Stap 6: Voorspel!
# voorspel()

# Generiek voor alle modellen
vanaf_datum_train_periode = '2019-01-01'
tot_datum_train_periode = '2023-05-15'
vanaf_datum_test_periode = '2023-05-15'
tot_datum_test_periode = '2026-01-01'

# Gemiddelde
# - Geen specifieke parameters

# Voortschrijdend_gemiddelde
window_size = 7
shift_period = 365
predict=False

# Lineaire regressie
yearly_seasonality=True
weekly_seasonality=True
transformation='spline'
n_bins=14
strategy= 'uniform'
n_knots= 4
degree= 4

df_toekomst = voorspel(
    df=df,
    onderwerp=onderwerp,
    vanaf_datum_train_periode = vanaf_datum_train_periode,
    tot_datum_train_periode = tot_datum_train_periode,
    vanaf_datum_test_periode = vanaf_datum_test_periode,
    tot_datum_test_periode = tot_datum_test_periode,
    window_size= window_size,
    shift_period = shift_period,
    predict = predict,
    yearly_seasonality = yearly_seasonality,
    weekly_seasonality = weekly_seasonality,
    transformation=transformation, n_bins=n_bins, strategy=strategy, n_knots=n_knots, degree=degree, shapes=False
)

# Belangrijke les om mee te nemen is het proces wat je opzet! 
# Dit proces is herhaalbaar en kan je steeds verbeteren. 
# Gebruik elke keer de resultaten van de vorige stap om de volgende stap te verbeteren.
# Corrigeer outliers indien bekend.

In [ ]:
# TODO:

# DONE: Fix shapes in plot functie (nu hardcoded 1 jaar van max)
# DONE: Verplaatsen functies naar preprocess, evaluate en model
# DONE: Requirements uitwerken
# Push naar github
# Test op colab
# Tekst toevoegen aan notebook
# Presentaie aanvullen met toelichting

# R2 toevoegen aan metrieken?
# Na workshop: Optimum uitrekenen voor elk dataset en model
# Code opschonen
# 